In [2]:
import time, enum, math
import numpy as np
import pandas as pd
import pylab as plt

from networkx.algorithms.shortest_paths.generic import has_path
import networkx as nx

import random
from tqdm import tqdm, trange

data_path = '../' #set to wherever the data files are, will be used on every input

## Data Imports

In [22]:
ports_df = pd.read_csv((data_path +'ports.csv'))
distances_df = pd.read_csv((data_path + 'distances.csv'))
origin = pd.read_csv((data_path + 'origin_ports.csv'))
route_blockages = pd.read_csv((data_path + 'route_blockage.csv'))
distances = distances_df[["prev_port", "next_port", "distance"]]
distances.astype({'prev_port':'int64', 'next_port':'int64'}).dtypes
N = distances_df["next_port"].tolist()
N = list(set(N))

In [36]:
G = nx.from_pandas_edgelist(distances, "prev_port", "next_port", ["distance"], create_using=nx.MultiGraph())


## Create Pruning Plans

When cutting the network, we want to ensure that we cut the correct number of edges 
(some port combinations have more than one edge connecting them, we assume that if one is blocked, all are blocked)

We therefore create a list indicating the number of edges, and then separate the file by 


In [40]:
"""
Commented out for your convenience, files are included
"""
# num_of_edges = []
# for index, row in route_blockages.iterrows(): 
#     num_of_edges.append(G.number_of_edges(u=route_blockages.iloc[index]['prev_port'], v=route_blockages.iloc[index]['next_port']))

# route_blockages["Num of edges"] = num_of_edges

# route_blockages_gib =  route_blockages[route_blockages['affected_by_gibraltar']==True]
# route_blockages_mal =  route_blockages[route_blockages['affected_by_malacca']==True]
# route_blockages_dov =  route_blockages[route_blockages['affected_by_dover']==True]
# route_blockages_suez =  route_blockages[route_blockages['affected_by_suez']==True]
# route_blockages_horm =  route_blockages[route_blockages['affected_by_hormuz']==True]
# route_blockages_pan = route_blockages[route_blockages['affected_by_panama']==True]

# route_blockages_horm = route_blockages_horm[["prev_port", "next_port", "Num of edges"]]
# route_blockages_gib =  route_blockages_gib[["prev_port", "next_port", "Num of edges"]]
# route_blockages_mal =  route_blockages_mal[["prev_port", "next_port", "Num of edges"]]
# route_blockages_dov =  route_blockages_dov[["prev_port", "next_port", "Num of edges"]]
# route_blockages_suez =  route_blockages_suez[["prev_port", "next_port", "Num of edges"]]
# route_blockages_pan = route_blockages_pan[["prev_port", "next_port", "Num of edges"]]

# route_blockages_horm.to_csv((data_path + 'route_blockages_horm.csv'))
# route_blockages_mal.to_csv((data_path + 'route_blockages_mal.csv'))
# route_blockages_dov.to_csv((data_path + 'route_blockages_dov.csv'))
# route_blockages_suez.to_csv((data_path + 'route_blockages_suez.csv'))
# route_blockages_gib.to_csv((data_path + 'route_blockages_gib.csv'))
# route_blockages_pan.to_csv((data_path + 'route_blockages_pan.csv'))

In [3]:
route_blockages_horm = pd.read_csv((data_path + 'route_blockages_horm.csv'))

In [ ]:
def Cut_Graph(G, route_blockages):
    print('Before Pruning',G.number_of_edges())
    for index in tqdm(range(len(route_blockages)), desc='Removing Edges', delay=0.5):
        for i in range(route_blockages.iloc[index]['Num of edges']):
            try:
                G.remove_edge(route_blockages.iloc[index]['prev_port'],route_blockages.iloc[index]['next_port'])
            except:
                pass
    print('\n', 'After Pruning',G.number_of_edges())
    return G

In [10]:
def Build_Cut_Graph(N, distances, route_blockages):
    G = nx.from_pandas_edgelist(distances, "prev_port", "next_port", ["distance"], create_using=nx.MultiGraph())
    # grid = NetworkGrid(G) #Define Mesa Grid as the just created Network to allow for shipping only in routes
    print('Before Pruning','Nodes: ',G.number_of_nodes(), 'Edges: ',G.number_of_edges())
    for index in tqdm(range(len(route_blockages)), desc='Removing Edges', delay=0.5):
        for i in range(route_blockages.iloc[index]['Num of edges']):
            try:
                G.remove_edge(route_blockages.iloc[index]['prev_port'],route_blockages.iloc[index]['next_port'])
            except:
                pass
    print('\n', 'After Pruning','Nodes: ',G.number_of_nodes(), 'Edges: ',G.number_of_edges())
    return G

In [11]:
G_without_Hormuz = Build_Cut_Graph(N, distances, route_blockages_horm)

Building Graph: 100%|██████████| 56154/56154 [00:19<00:00, 2842.39it/s]
Before Pruning Nodes:  2912 Edges:  56154
Removing Edges: 100%|██████████| 1156/1156 [00:00<00:00, 2173.09it/s]
 After Pruning Nodes:  2912 Edges:  54481



1673